## CBIR CON FAST

In [1]:
import tensorflow as tf

import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
import cv2
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Función para calcular los descriptores FAST de una imagen
def calculate_fast_descriptors(image, max_keypoints=100):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Crear el detector FAST
    fast = cv2.FastFeatureDetector_create()

    # Encontrar los keypoints
    keypoints = fast.detect(gray_image, None)

    # Tomar un máximo de 'max_keypoints' keypoints
    if len(keypoints) > max_keypoints:
        keypoints = sorted(keypoints, key=lambda x: -x.response)[:max_keypoints]

    if not keypoints:
        return [], np.zeros((0, 32), dtype=np.uint8)  # Devolver listas vacías o descriptores de ceros si no hay keypoints

    # Calcular los descriptores BRIEF para los keypoints
    brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
    keypoints, descriptors = brief.compute(gray_image, keypoints)

    if descriptors is None or descriptors.shape[1] != 32:
        return keypoints, np.zeros((0, 32), dtype=np.uint8)  # Devolver keypoints y descriptores de ceros si no hay descriptores o no tienen 32 características

    return keypoints, descriptors


# Cargar la imagen de consulta
query_image = cv2.imread('n04487081_3.JPEG')
query_keypoints, query_descriptors = calculate_fast_descriptors(query_image)

# Crear listas de keypoints y descriptores FAST para la base de datos
database_images = []
for i in range(100):
    image_path = f'n04487081_{i}.JPEG'
    image = cv2.imread(image_path)
    database_keypoints, database_descriptors = calculate_fast_descriptors(image)
    database_images.append((image, database_keypoints, database_descriptors))

# Crear un modelo de k-NN
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')  # Puedes ajustar el número de vecinos (n_neighbors) y la métrica según tus necesidades

# Extraer los descriptores de la base de datos
database_descriptors = [item[2] for item in database_images]

# Filtrar los descriptores vacíos (None)
database_descriptors = [descriptor for adescriptor in database_descriptors if descriptor.any()]

# Verificar si hay descriptores en la base de datos
if not database_descriptors:
    print("No se detectaron descriptores en la base de datos. Asegúrate de que haya suficientes keypoints en las imágenes de la base de datos.")
else:
    # Encontrar la longitud máxima de los descriptores en la base de datos
    max_descriptor_length = max(len(descriptor) for descriptor in database_descriptors)

    # Rellenar los descriptores más cortos con ceros
    for i in range(len(database_descriptors)):
        descriptor = database_descriptors[i]
        if len(descriptor) < max_descriptor_length:
            descriptor = np.pad(descriptor, (0, max_descriptor_length - len(descriptor)), 'constant')
        database_descriptors[i] = descriptor

    # Apilar los descriptores
    database_descriptors = np.vstack(database_descriptors)

    # Ajustar el modelo k-NN a los descriptores de la base de datos
    knn.fit(database_descriptors)

    # Calcular los descriptores para la imagen de consulta
    query_descriptors = calculate_fast_descriptors(query_image)[1]

    # Encontrar las imágenes más similares usando k-NN
    distances, indices = knn.kneighbors(query_descriptors)

    # Mostrar las imágenes recuperadas más similares
    display_n_similar(indices[0], 8)


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
